In [ ]:
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import scipy.io
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape, Dropout, BatchNormalization, Input, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from tensorflow.keras.optimizers import Adam
from ultralytics import YOLO

In [ ]:
mat_file_path = '/Users/Study/Downloads/WIDER/wider_face_split/wider_face_train.mat'
mat_train = scipy.io.loadmat(mat_file_path)

print(mat_train.keys())

In [ ]:
file_list = mat_train['file_list']
face_list = mat_train['face_bbx_list']

In [ ]:
names_bboxes = {}

for index in range(len(file_list)):
  counter = -1
  for img_obj in file_list[index][0]:
    counter += 1
    params_list = []
    for params in face_list[index][0][counter][0]:
      params[0] += params[2]/2
      params[1] += params[3]/2
      params_list.append(np.array(params))
    names_bboxes[img_obj[0][0]] = (params_list)

In [ ]:
names_images_dims = [[], [], []]

base_path = '/Users/Study/Downloads/WIDER/WIDER_train_full'

for img_name in os.listdir(base_path):
  img_path = os.path.join(base_path, img_name)
  img = cv2.imread(img_path)
  if img is not None:
    original_height, original_width = img.shape[:2]
    coeff = original_height/original_width
    base = 240
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (int(base), int(base*coeff)))
    names_images_dims[0].append(img_name[:-4])
    names_images_dims[1].append(img/255.0)
    names_images_dims[2].append([float(original_width), float(original_height)])

In [ ]:
images_bboxes_train = {}
for name in names_images_dims[0]:
  images_bboxes_train[name] = names_images_dims[1][names_images_dims[0].index(name)], names_bboxes[name], names_images_dims[2][names_images_dims[0].index(name)]

In [ ]:
for key in images_bboxes_train.keys():
  counter = -1
  for faces in images_bboxes_train[key][1]:
    counter+=1
    images_bboxes_train[key][1][counter] = images_bboxes_train[key][1][counter].astype(np.float32)
    images_bboxes_train[key][1][counter][0] = faces[0]/images_bboxes_train[key][2][0]
    images_bboxes_train[key][1][counter][1] = faces[1]/images_bboxes_train[key][2][1]
    images_bboxes_train[key][1][counter][2] = faces[2]/images_bboxes_train[key][2][0]
    images_bboxes_train[key][1][counter][3] = faces[3]/images_bboxes_train[key][2][1]

In [ ]:
keys = list(images_bboxes_train.keys())

In [ ]:
val_dict = {}
for key in range(150):
  val_dict[keys[key]] = images_bboxes_train[keys[key]]
  images_bboxes_train.pop(keys[key])

In [ ]:
keys = list(images_bboxes_train.keys())

In [ ]:
for key in range(10000):
  images_bboxes_train.pop(keys[key])

In [ ]:
imroot = '/Users/Study/Downloads/WIDER/root/images/train/'
lbroot = '/Users/Study/Downloads/WIDER/root/labels/train/'

for key in images_bboxes_train.keys():
  file_path = os.path.join(imroot, f"{key}.jpg")
  image = np.clip(images_bboxes_train[key][0] * 255.0, 0, 255).astype(np.uint8)
  cv2.imwrite(file_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

for key, values in images_bboxes_train.items():
  file_path = os.path.join(lbroot, f"{key}.txt")
  with open(file_path, 'w') as file:
    for bbox in values[1]:
      bbox_str = "0 " + " ".join(map(str, bbox))
      file.write(bbox_str + "\n")  

In [ ]:
imroot = '/Users/Study/Downloads/WIDER/root/images/val/'
lbroot = '/Users/Study/Downloads/WIDER/root/labels/val/'

for key in val_dict.keys():
  file_path = os.path.join(imroot, f"{key}.jpg")
  image = np.clip(val_dict[key][0] * 255.0, 0, 255).astype(np.uint8)
  cv2.imwrite(file_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

for key, values in val_dict.items():
  file_path = os.path.join(lbroot, f"{key}.txt")
  with open(file_path, 'w') as file:
    for bbox in values[1]:
      bbox_str = "0 " + " ".join(map(str, bbox))
      file.write(bbox_str + "\n") 

In [ ]:
model = YOLO("yolov8n.yaml")

In [ ]:
res = model.train(data='faces.yaml', verbose=True, epochs=10)